## 14. Estimating the Spectrum

**Exercise 14.1**. Consider the variance of $\overline{S}(f)$ with the Daniell spectral window. Instead of using Equation (14.2.4) on page 355, use the fact that $2 \hat{S}(f) / S(f)$ has approximately a chi-square distribution with two degrees of freedom to show that the smoothed sample spectral density has an approximate variance of $S^2(f) / (2m + 1)$.

**Solution**.  The smoothed estimator of the spectral density is

$$ \overline{S}(f) = \sum_{k=-m}^m W_m(k) \hat{S}\left( f + \frac{k}{n} \right) \tag{14.1.3} $$

and the Daniell spectral window corresponds to weights

$$ W_m(k) = \frac{1}{2m + 1} \quad \text{for } -m \leq k \leq m \tag{14.1.4} $$

Therefore, using this smoothing window,

$$ \frac{\overline{S}(f)}{S(f)} = \frac{1}{2(2m + 1)} \sum_{k=-m}^m \frac{2 \hat{S}\left( f + \frac{k}{n} \right)}{S(f)} $$

Taking the variance,

$$ 
\begin{align}
\text{Var}\left[ \frac{\overline{S}(f)}{S(f)} \right] 
&= \frac{1}{4(2m+1)^2} \text{Var}\left[ \sum_{k=-m}^m \frac{2 \hat{S}\left( f + \frac{k}{n} \right)}{S(f)} \right] \\
&\approx \frac{1}{4(2m+1)^2} \sum_{k=-m}^m  \text{Var}\left[ \frac{2 \hat{S}\left( f + \frac{k}{n} \right)}{S(f)} \right] \\
&\approx \frac{1}{4(2m+1)^2} \sum_{k=-m}^m 2 \cdot 2 \\
&= \frac{1}{2m + 1}
\end{align}
$$

since the spectral sample density values at Fourier frequencies are approximately uncorrelated, and since the variance of a chi-square variable with 2 degrees of freedom is $2 \cdot \text{df} = 4$.

Therefore, taking $S(f)$ out as a constant factor, we obtain the result

$$ \text{Var}\left[\overline{S}(f)\right] \approx \frac{S^2(f)}{2m + 1} $$

**Exercise 14.2**. Consider various convolutions of the simple Daniell rectangular spectral window.

**(a)** Construct a panel of three plots similar to those shown in Exhibit 14.3 on page 354 but with the Daniell spectral window and with $m = 5$. The middle graph should be the convolution of two Daniell windows and the leftmost graph the convolution of three Daniell windows.

**(b)** Evaluate the bandwidths and degrees of freedom for each of the spectral windows constructed in part (a). Use $n = 100$.

**(c)** Construct another panel of three plots similar to those shown in Exhibit 14.3 but with the modified Daniell spectral window. This time use $m = 5$ for the first graph and convolve two with $m = 5$ and $m = 7$ for the second. Convolve three windows with $m$’s of 5, 7, and 11 for the third graph.

**(d)** Evaluate the bandwidths and degrees of freedom for each of the spectral windows constructed in part (c). Use $n = 100$.

**Exercise 14.3**. For the Daniell rectangular spectral window show that

**(a)** 

$$ \frac{1}{n^2} \sum_{k=-m}^m k^2 W_m(k) = \frac{2}{n^2(2m+1)} \left( \frac{m^3}{3} + \frac{m^2}{2} + \frac{m}{6} \right) $$

**(b)** Show that if $m$ is chosen as $m = c\sqrt{n}$ for any constant $c$, then the right-hand side of the expression in part (a) tends to zero as $n$ goes to infinity.

**(c)** Show that if $m = c\sqrt{n}$ for any constant $c$, then the approximate variance of the smoothed spectral density given by the right-hand side of Equation (14.2.4) on page 355 tends to zero as n tends to infinity.

**Exercise 14.4**.  Suppose that the distribution of $\overline{S}(f)$ is to be approximated by a multiple of a chi-square variable with degrees of freedom $\nu$, so that $\overline{S}(f) \approx c \chi_\nu^2$. Using the approximate variance of given in Equation (14.2.4) on page 355 and the fact that $\overline{S}(f)$ is approximately unbiased, equate means and variances and find the values for $c$ and $\nu$ (thus establishing Equation (14.4.2) on page 356).

**Exercise 14.5**.  Construct a time series of length $n = 48$ according to the expression 

$$ Y_t = \sin \left[ 2 \pi (0.28) t \right] $$

Display the periodogram of the series and explain its appearance.

**Exercise 14.6**.  Estimate the spectrum of the Los Angeles annual rainfall time series. The data are in the file named `larain`. Because of the skewness in the series, use the logarithms of the raw rainfall values. The square root of the series length suggests a value for the *span* of about 11. Use the modified Daniell spectral window, and be sure to set the vertical limits of the plot so that you can see the whole confidence interval guide. Comment on the estimated spectrum.

**Exercise 14.7**. The file named `spots1` contains annual sunspot numbers for 306 years from 1700 through 2005.

**(a)** Display the time series plot of these data. Does stationarity seem reasonable for this series?

**(b)** Estimate the spectrum using a modified Daniell spectral window convoluted with itself and a *span* of 3 for both. Interpret the plot.

**(c)** Estimate the spectrum using an AR model with the order chosen to minimize the AIC. Interpret the plot. What order was selected?

**(d)** Overlay the estimates obtained in parts (b) and (c) above onto one plot. Do they agree to a reasonable degree?

**Exercise 14.8**. Consider the time series of average monthly temperatures in Dubuque, Iowa. The data are in the file named `tempdub` and cover from January 1964 to December 1975 for an $n$ of 144.

**(a)** Estimate the spectrum using a variety of span values for the modified Daniell spectral window.

**(b)** In your opinion, which of the estimates in part (a) best represents the spectrum of the process? Be sure to use bandwidth considerations and confidence limits to back up your argument.

**Exercise 14.9**. An EEG (electroencephalogram) time series is given in the data file named `eeg`. An electroencephalogram is a noninvasive test used to detect and record the electrical activity generated in the brain. These data were measured at a sampling rate of 256 per second and came from a patient suffering a seizure. The total record length is $n = \text{13,000}$ -- or slightly less than one minute.

**(a)** Display the time series plot and decide if stationarity seems reasonable.

**(b)** Estimate the spectrum using a modified Daniell spectral window convolved with itself and a *span* of 51 for both components of the convolution. Interpret the plot.

**(c)** Estimate the spectrum using an AR model with the order chosen to minimize the AIC. Interpret the plot. What order was selected?

**(d)** Overlay the estimates obtained in parts (b) and (c) above onto one plot. Do they agree to a reasonable degree?

**Exercise 14.10**. The file named `electricity` contains monthly U. S. electricity production values from January 1994 to December 2005. A time series plot of the logarithms of these values is shown in Exhibit 11.14 on page 264. Since there is an upward trend and increasing variability at higher levels in these data, use the first difference of the logarithms for the remaining analysis.

**(a)** Construct a time series plot of the first difference of the logarithms of the electricity values. Does a stationary model seem warranted at this point?

**(b)** Display the smoothed spectrum of the first difference of the logarithms using a modified Daniell spectral window and *span* values of 25, 13, and 7. Interpret the results.

**(c)** Now use a spectral window that is a convolution of two modified Daniell windows each with *span* $= 3$. Also use a 10% taper. Interpret the results.

**(d)** Estimate the spectrum using an AR model with the order chosen to minimize the AIC. Interpret the plot. What order was selected?

**(e)** Overlay the estimates obtained in parts (c) and (d) above onto one plot. Do they agree to a reasonable degree?